In [ ]:
with open('adult.data','r')as f:
    traindata=[[p.strip() for  p in x.strip('\n').strip('.').split(',')] for x in f.readlines()]

In [ ]:
with open('adult.test','r')as f:
    testdata=[[p.strip() for  p in x.strip('\n').strip('.').split(',')] for x in f.readlines()]

In [ ]:
datadict={i:{} for i in range(15)}

for data in traindata+testdata:
    for i in range(15):
        if data[i] not in datadict[i]:
            datadict[i][data[i]]=len(datadict[i])


In [ ]:
from keras.utils.np_utils import to_categorical
cateid=[1,3,5,6,7,8,9,13]
alldata=[]
alllabel=[]
allgen=[]
for data in traindata+testdata:
    tp=[]
    for i in range(14):
        if i==9:
            allgen.append(datadict[i][data[i]])
            continue
        if i in cateid:
            tp+=to_categorical(datadict[i][data[i]],len(datadict[i])).tolist()
        else:
            tp.append(float(data[i]))
    
    alldata.append(tp)
    alllabel.append(datadict[14][data[14]])

In [ ]:
import numpy as np
alldata=np.array(alldata)
alllabel=np.array(alllabel)
allgen=np.array(allgen)
for i in range(len(alldata[0])):
    alldata[:,i]/=np.max(alldata[:,i])
meandata=np.mean(alldata,axis=0)

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import *

# Notes: on some versions of sklearn, the LR model does not converge for the first task. To be fixed in the future.

result_1=[]
result_2=[]
result_3=[]
deposit=0.05
#federated LR has almost the same performance with standard LR
for i in range(5): 
    
    t2=np.random.uniform()*deposit
    t3=np.random.uniform()*deposit 
    samplemask2=np.array(np.random.uniform(size=(len(alldata),))<t2,dtype='int32')
    s2d=np.expand_dims((1-samplemask2),axis=1)* np.ones_like(alldata)*meandata+np.expand_dims(samplemask2,axis=1)*alldata
    samplemask3=np.array(np.random.uniform(size=(len(alldata),))<t3,dtype='int32')
    s3d=np.expand_dims((1-samplemask3),axis=1)* np.ones_like(alldata)*meandata+np.expand_dims(samplemask3,axis=1)*alldata 
    reg = LogisticRegression(class_weight='balanced',max_iter=1000).fit(np.concatenate([alldata[:len(traindata),:11],s2d[:len(traindata),28: 61 ],s3d[:len(traindata),61: ]],axis=-1),  np.argmax(alldata[:len(traindata),12:27],axis=-1))
    pred=reg.predict(np.concatenate([alldata[len(traindata):,:11],s2d[len(traindata):,28: 61 ],s3d[len(traindata):,61:  ]],axis=-1)) 

    a1=accuracy_score(pred, np.argmax(alldata[len(traindata):,12:27],axis=-1))
    print(a1)
    result_1.append([t2,t3,a1])
    
    
    t1=np.random.uniform()*deposit
    t3=np.random.uniform()*deposit
    samplemask1=np.array(np.random.uniform(size=(len(alldata),))<t1,dtype='int32')
    s1d=np.expand_dims((1-samplemask1),axis=1)* np.ones_like(alldata)*meandata+np.expand_dims(samplemask1,axis=1)*alldata
    samplemask3=np.array(np.random.uniform(size=(len(alldata),))<t3,dtype='int32')
    s3d=np.expand_dims((1-samplemask3),axis=1)* np.ones_like(alldata)*meandata+np.expand_dims(samplemask3,axis=1)*alldata   
    reg = LogisticRegression().fit(np.concatenate([s1d[:len(traindata),:11],alldata[:len(traindata),28: 61 ],s3d[:len(traindata),61: ]],axis=-1), allgen[:len(traindata)])
    pred=reg.predict(np.concatenate([s1d[len(traindata):,:11],alldata[len(traindata):,28: 61 ],s3d[len(traindata):,61:  ]],axis=-1)) 

    a2=accuracy_score(pred,allgen[len(traindata):])
    result_2.append([t1,t3,a2])
        
    t1=np.random.uniform()*deposit
    t2=np.random.uniform()*deposit
    samplemask1=np.array(np.random.uniform(size=(len(alldata),))<t1,dtype='int32')
    s1d=np.expand_dims((1-samplemask1),axis=1)* np.ones_like(alldata)*meandata+np.expand_dims(samplemask1,axis=1)*alldata
    samplemask2=np.array(np.random.uniform(size=(len(alldata),))<t2,dtype='int32')
    s2d=np.expand_dims((1-samplemask2),axis=1)* np.ones_like(alldata)*meandata+np.expand_dims(samplemask2,axis=1)*alldata 
    reg = LogisticRegression().fit(np.concatenate([s1d[:len(traindata),:11],s2d[:len(traindata),28: 61 ],alldata[:len(traindata),61: ]],axis=-1), alllabel[:len(traindata)])
    pred=reg.predict(np.concatenate([s1d[len(traindata):,:11],s2d[len(traindata):,28: 61 ],alldata[len(traindata):,61:  ]],axis=-1)) 

    a3=accuracy_score(pred,alllabel[len(traindata):])
    result_3.append([t1,t2,a3])
   

In [ ]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn import linear_model


def stdError_func(y_test, y):
    return np.sqrt(np.mean((y_test - y) ** 2))
def R2_1_func(y_test, y):
    return 1 - ((y_test - y) ** 2).sum() / ((y.mean() - y) ** 2).sum()

def R2_2_func(y_test, y):
    y_mean = np.array(y)
    y_mean[:] = y.mean()
    return 1 - stdError_func(y_test, y) / stdError_func(y_mean, y) 

def regression(result):
    result=np.array(result)
    x = np.array(result[:,0:2])
    y = np.array(result[:,2] )
    poly_reg =PolynomialFeatures(degree=1) 
    X_ploy =poly_reg.fit_transform(x)
    lin_reg=linear_model.LinearRegression()
    lin_reg.fit(X_ploy,y)
    predict_y =  lin_reg.predict(X_ploy)
    R2_1 = R2_1_func(predict_y, y)
    R2_2 = R2_2_func(predict_y, y)
    print("coefficients", lin_reg.coef_)
    print("intercept", lin_reg.intercept_)
    return lin_reg

lin_reg_1 = regression(result_1)
lin_reg_2 = regression(result_2)
lin_reg_3 = regression(result_3)

In [ ]:
state1=np.random.uniform(0,1,(3,))
state1/=np.sum(state1)
state2=np.random.uniform(0,1,(3,))
state2/=np.sum(state2)
state3=np.random.uniform(0,1,(3,))
state3/=np.sum(state3)
lr=0.1
gamma=2.5
eps=1e-8
_,w21,w31=lin_reg_1.coef_
_,w12,w32=lin_reg_2.coef_
_,w13,w23=lin_reg_3.coef_
for _round in range(1000000):
    x11,x12,x13=state1+eps
    x21,x22,x23=state2+eps
    x31,x32,x33=state3+eps
  
    r3=x13*w13+x23*w23
    r2=x12*w12+x32*w32
    r1=x21*w21+x31*w31
    
    d3=np.array([-np.power(w13*x13,gamma)/((x31+eps)**2), -1/((x32+eps)**2)*np.power(w23*x23,gamma), 0.])
    d2=np.array([-np.power(x12*w12,gamma)/((x21+eps)**2),0.,-1/((x23+eps)**2)*np.power(w32*x32,gamma)])
    d1=np.array([0,-np.power(x21*w21,gamma)/((x12+eps)**2),-1/((x13+eps)**2)*np.power(w31*x31,gamma)])         
    state1-=lr*d1
    state2-=lr*d2
    state3-=lr*d3
    
    state1[0]=0
    if np.sum(state1)>1:
        state1/=np.sum(state1)
        
    state1=np.maximum(state1, 0)    

    state2=np.maximum(state2, 0)
    state2[1]=0
    if np.sum(state2)>1:
        state2/=np.sum(state2) 

    state3=np.maximum(state3, 0)
    state3[2]=0
    if np.sum(state3)>1:
        state3/=np.sum(state3)
    if rounder %5000==0:
        print(r1,r2,r3,r1+r2+r3)
        print(state1,state2,state3)

In [ ]:
t2=state1[1] 
t3=state1[2] 
samplemask1=np.array(np.random.uniform(size=(len(alldata),))<t2,dtype='int32')
s2d=np.expand_dims((1-samplemask1),axis=1)* np.ones_like(alldata)*meandata+np.expand_dims(samplemask1,axis=1)*alldata
samplemask3=np.array(np.random.uniform(size=(len(alldata),))<t3,dtype='int32')
s3d=np.expand_dims((1-samplemask3),axis=1)* np.ones_like(alldata)*meandata+np.expand_dims(samplemask3,axis=1)*alldata 
reg = LogisticRegression(class_weight='balanced').fit(np.concatenate([alldata[:len(traindata),:11],s2d[:len(traindata),28: 61 ],s3d[:len(traindata),61: ]],axis=-1),  np.argmax(alldata[:len(traindata),12:27],axis=-1))
pred=reg.predict(np.concatenate([alldata[len(traindata):,:11],s2d[len(traindata):,28: 61 ],s3d[len(traindata):,61:  ]],axis=-1)) 
print(accuracy_score(pred, np.argmax(alldata[len(traindata):,12:27],axis=-1))) 

t1=state2[0]
t3=state2[2]
samplemask1=np.array(np.random.uniform(size=(len(alldata),))<t1,dtype='int32')
s1d=np.expand_dims((1-samplemask1),axis=1)* np.ones_like(alldata)*meandata+np.expand_dims(samplemask1,axis=1)*alldata
samplemask3=np.array(np.random.uniform(size=(len(alldata),))<t3,dtype='int32')
s3d=np.expand_dims((1-samplemask3),axis=1)* np.ones_like(alldata)*meandata+np.expand_dims(samplemask3,axis=1)*alldata   
reg = LogisticRegression().fit(np.concatenate([s1d[:len(traindata),:11],alldata[:len(traindata),28: 61 ],s3d[:len(traindata),61: ]],axis=-1), allgen[:len(traindata)])
pred=reg.predict(np.concatenate([s1d[len(traindata):,:11],alldata[len(traindata):,28: 61 ],s3d[len(traindata):,61:  ]],axis=-1)) 
print(accuracy_score(pred,allgen[len(traindata):]))

t1=state3[0] 
t2=state3[1] 
samplemask1=np.array(np.random.uniform(size=(len(alldata),))<t1,dtype='int32')
s1d=np.expand_dims((1-samplemask1),axis=1)* np.ones_like(alldata)*meandata+np.expand_dims(samplemask1,axis=1)*alldata
samplemask2=np.array(np.random.uniform(size=(len(alldata),))<t3,dtype='int32')
s2d=np.expand_dims((1-samplemask2),axis=1)* np.ones_like(alldata)*meandata+np.expand_dims(samplemask2,axis=1)*alldata 
reg = LogisticRegression().fit(np.concatenate([s1d[:len(traindata),:11],s2d[:len(traindata),28: 61 ],alldata[:len(traindata),61: ]],axis=-1), alllabel[:len(traindata)])
pred=reg.predict(np.concatenate([s1d[len(traindata):,:11],s2d[len(traindata):,28: 61 ],alldata[len(traindata):,61:  ]],axis=-1)) 
print(accuracy_score(pred,alllabel[len(traindata):]))